# Resource Manager

> Track worker processes and detect resource conflicts for GPU/CPU usage

In [ ]:
#| default_exp core.manager

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Set, TYPE_CHECKING
from enum import Enum
from dataclasses import dataclass, field

# Import from cjm-fasthtml-plugins for enhanced plugin support
if TYPE_CHECKING:
    from cjm_fasthtml_plugins.core.execution_mode import PluginExecutionMode
    from cjm_fasthtml_plugins.core.metadata import RemoteResourceInfo

## Resource Types and Status

The resource manager tracks different types of resources (GPU memory, system memory) and their availability status.

In [ ]:
#| export
class ResourceType(Enum):
    """Types of resources to monitor."""
    GPU_MEMORY = "gpu_memory"
    SYSTEM_MEMORY = "system_memory"

In [ ]:
#| export
class ResourceStatus(Enum):
    """Status of resource availability."""
    AVAILABLE = "available"
    APP_BUSY = "app_busy"  # Resource in use by this application
    EXTERNAL_BUSY = "external_busy"  # Resource in use by external process

## Data Structures

Data classes for tracking resource conflicts and worker states.

In [ ]:
#| export
@dataclass
class ResourceConflict:
    """Information about a resource conflict."""
    resource_type: ResourceType
    status: ResourceStatus
    app_pids: List[int]  # PIDs from this application using the resource
    external_pids: List[int]  # External PIDs using the resource
    app_processes: List[Dict[str, Any]]  # Detailed info about app processes
    external_processes: List[Dict[str, Any]]  # Detailed info about external processes

In [ ]:
#| export
@dataclass
class WorkerState:
    """State of a worker process.
    
    Enhanced to support lifecycle-aware and cloud-aware plugins from cjm-fasthtml-plugins.
    """
    pid: int
    worker_type: str  # e.g., "transcription", "llm"
    job_id: Optional[str] = None
    plugin_id: Optional[str] = None
    plugin_name: Optional[str] = None
    loaded_plugin_resource: Optional[str] = None  # The plugin resource identifier currently loaded
    config: Optional[Dict[str, Any]] = None  # Current plugin configuration
    status: str = "idle"  # idle, running, busy
    
    # Extended for lifecycle-aware plugins (from cjm-fasthtml-plugins)
    execution_mode: Optional[str] = None  # Execution mode (in_process, subprocess, cloud_gpu, etc.)
    child_pids: List[int] = field(default_factory=list)  # PIDs of child processes
    container_id: Optional[str] = None  # Docker container ID if applicable
    conda_env: Optional[str] = None  # Conda environment name if applicable
    
    # Extended for cloud-aware plugins
    is_remote: bool = False  # Whether this worker uses remote/cloud resources
    remote_resource: Optional[Dict[str, Any]] = None  # Remote resource info (serialized RemoteResourceInfo)

In [ ]:
# Example: Creating a worker state
worker = WorkerState(
    pid=12345,
    worker_type="transcription",
    plugin_name="whisper",
    status="idle"
)

print(f"Worker PID: {worker.pid}")
print(f"Worker type: {worker.worker_type}")
print(f"Plugin: {worker.plugin_name}")
print(f"Status: {worker.status}")

Worker PID: 12345
Worker type: transcription
Plugin: whisper
Status: idle


### Enhanced Worker State

The `WorkerState` dataclass now supports lifecycle-aware and cloud-aware plugins:
- **Lifecycle fields**: `execution_mode`, `child_pids`, `container_id`, `conda_env`
- **Cloud fields**: `is_remote`, `remote_resource`

These fields are optional and only used when working with plugins from `cjm-fasthtml-plugins` that implement the `LifecycleAwarePlugin` or `CloudAwarePlugin` protocols.

## Configuration Keys

Common configuration keys that indicate the plugin resource being used.

In [ ]:
#| export
# Common configuration keys that indicate the plugin resource being used
PLUGIN_RESOURCE_CONFIG_KEYS = ["resource_id", "model_id", "model", "model_name", "model_path"]

## ResourceManager

The `ResourceManager` class tracks worker processes and provides methods to check resource availability and detect conflicts.

In [ ]:
#| export
class ResourceManager:
    """Manages resource tracking and conflict detection for the application. Tracks PIDs associated with application workers (transcription, LLM, etc.) and provides methods to check resource availability and conflicts. Enhanced to support lifecycle-aware and cloud-aware plugins from cjm-fasthtml-plugins."""

    def __init__(
        self,
        gpu_memory_threshold_percent:float=45.0 # GPU memory usage threshold; external processes using more than this percentage are considered conflicts
    ):
        """Initialize the resource manager."""
        self._worker_states: Dict[int, WorkerState] = {}  # pid -> WorkerState
        self._job_to_pid: Dict[str, int] = {}  # job_id -> pid
        self._child_to_parent: Dict[int, int] = {}  # child_pid -> parent_pid
        self.gpu_memory_threshold_percent = gpu_memory_threshold_percent

    def register_worker(
        self,
        pid:int, # Process ID of the worker
        worker_type:str, # Type of worker (e.g., "transcription", "llm")
        job_id:Optional[str]=None, # Optional job ID if worker is processing a job
        plugin_id:Optional[str]=None, # Optional plugin unique ID
        plugin_name:Optional[str]=None, # Optional plugin name
        loaded_plugin_resource:Optional[str]=None, # Optional identifier of the loaded plugin resource
        config:Optional[Dict[str, Any]]=None, # Optional plugin configuration
        plugin_instance:Optional[Any]=None # Optional plugin instance for lifecycle/cloud protocol detection
    ):
        """Register a worker process with the resource manager."""
        # Create base worker state
        worker = WorkerState(
            pid=pid,
            worker_type=worker_type,
            job_id=job_id,
            plugin_id=plugin_id,
            plugin_name=plugin_name,
            loaded_plugin_resource=loaded_plugin_resource,
            config=config,
            status="idle"
        )
        
        # If plugin instance provided, check for lifecycle/cloud awareness
        if plugin_instance:
            try:
                from cjm_fasthtml_plugins.protocols.lifecycle import is_lifecycle_aware
                from cjm_fasthtml_plugins.protocols.cloud_aware import is_cloud_aware
                
                # Check lifecycle awareness
                if is_lifecycle_aware(plugin_instance):
                    worker.execution_mode = plugin_instance.get_execution_mode().value
                    worker.child_pids = plugin_instance.get_child_pids()
                    
                    # Track child -> parent mapping
                    for child_pid in worker.child_pids:
                        self._child_to_parent[child_pid] = pid
                    
                    # Get managed resources info
                    resources = plugin_instance.get_managed_resources()
                    if 'container_id' in resources:
                        worker.container_id = resources['container_id']
                    if 'conda_env' in resources:
                        worker.conda_env = resources['conda_env']
                
                # Check cloud awareness
                if is_cloud_aware(plugin_instance):
                    remote_info = plugin_instance.get_remote_resource_info()
                    if remote_info:
                        worker.is_remote = True
                        # Serialize RemoteResourceInfo to dict for storage
                        worker.remote_resource = {
                            'provider': remote_info.provider.value if hasattr(remote_info.provider, 'value') else str(remote_info.provider),
                            'region': remote_info.region,
                            'instance_id': remote_info.instance_id,
                            'status': remote_info.status,
                            'ssh_host': remote_info.ssh_host,
                            'gpu_count': remote_info.gpu_count,
                            'gpu_type': remote_info.gpu_type,
                            'estimated_cost_per_hour': remote_info.estimated_cost_per_hour
                        }
            except ImportError:
                # cjm-fasthtml-plugins not available, skip protocol checks
                pass
        
        self._worker_states[pid] = worker

        if job_id:
            self._job_to_pid[job_id] = pid
    
    def get_all_related_pids(
        self,
        parent_pid:int # Parent worker PID
    ) -> List[int]: # List of all PIDs (parent + children)
        """Get parent PID and all child PIDs managed by this worker."""
        worker = self._worker_states.get(parent_pid)
        if not worker:
            return [parent_pid]
        return [parent_pid] + worker.child_pids

    def update_worker_state(
        self,
        pid:int, # Process ID of the worker
        job_id:Optional[str]=None, # Optional job ID to update
        plugin_id:Optional[str]=None, # Optional plugin ID to update
        plugin_name:Optional[str]=None, # Optional plugin name to update
        loaded_plugin_resource:Optional[str]=None, # Optional loaded plugin resource to update
        config:Optional[Dict[str, Any]]=None, # Optional config to update
        status:Optional[str]=None # Optional status to update
    ):
        """Update the state of a registered worker."""
        if pid not in self._worker_states:
            return

        worker = self._worker_states[pid]

        if job_id is not None:
            worker.job_id = job_id
            if job_id:
                self._job_to_pid[job_id] = pid

        if plugin_id is not None:
            worker.plugin_id = plugin_id

        if plugin_name is not None:
            worker.plugin_name = plugin_name

        if loaded_plugin_resource is not None:
            worker.loaded_plugin_resource = loaded_plugin_resource

        if config is not None:
            worker.config = config

        if status is not None:
            worker.status = status

    def unregister_worker(
        self,
        pid:int # Process ID of the worker to unregister
    ):
        """Unregister a worker process."""
        if pid in self._worker_states:
            worker = self._worker_states[pid]
            
            # Clean up child PID mappings
            for child_pid in worker.child_pids:
                if child_pid in self._child_to_parent:
                    del self._child_to_parent[child_pid]
            
            if worker.job_id and worker.job_id in self._job_to_pid:
                del self._job_to_pid[worker.job_id]
            del self._worker_states[pid]

    def get_worker_by_pid(
        self,
        pid:int # Process ID
    ) -> Optional[WorkerState]: # Worker state or None
        """Get worker state by PID."""
        return self._worker_states.get(pid)

    def get_worker_by_job(
        self,
        job_id:str # Job ID
    ) -> Optional[WorkerState]: # Worker state or None
        """Get worker state by job ID."""
        pid = self._job_to_pid.get(job_id)
        if pid:
            return self._worker_states.get(pid)
        return None

    def get_all_workers(self) -> List[WorkerState]: # List of all registered workers
        """Get all registered workers."""
        return list(self._worker_states.values())

    def get_app_pids(self) -> Set[int]: # Set of all PIDs managed by this application (parents only)
        """Get all PIDs managed by this application (parents only)."""
        return set(self._worker_states.keys())
    
    def get_all_app_pids_including_children(self) -> Set[int]: # Set of all PIDs (parents and children)
        """Get all PIDs managed by this application including child processes."""
        all_pids = set(self._worker_states.keys())
        for worker in self._worker_states.values():
            all_pids.update(worker.child_pids)
        return all_pids

    def get_workers_by_type(
        self,
        worker_type:str # Type of worker (e.g., "transcription", "llm", "ollama")
    ) -> List[WorkerState]: # List of workers matching the type
        """Get all workers of a specific type."""
        return [w for w in self._worker_states.values() if w.worker_type == worker_type]

    def get_active_worker_types(self) -> Set[str]: # Set of worker type strings
        """Get set of all active worker types."""
        return {w.worker_type for w in self._worker_states.values()}

    def has_worker_type(
        self,
        worker_type:str # Type of worker to check
    ) -> bool: # True if at least one worker of this type exists
        """Check if a worker of the specified type exists."""
        return any(w.worker_type == worker_type for w in self._worker_states.values())
    
    def get_cloud_workers(self) -> List[WorkerState]: # List of workers with is_remote=True
        """Get all workers using cloud/remote resources."""
        return [w for w in self._worker_states.values() if w.is_remote]
    
    def estimate_total_cloud_cost(
        self,
        duration_hours:float=1.0 # Duration to estimate for
    ) -> float: # Total estimated cost in USD
        """Estimate total cost of all running cloud resources."""
        total = 0.0
        for worker in self.get_cloud_workers():
            if worker.remote_resource and worker.remote_resource.get('estimated_cost_per_hour'):
                total += worker.remote_resource['estimated_cost_per_hour'] * duration_hours
        return total

    def check_gpu_availability(self) -> ResourceConflict: # ResourceConflict with details about GPU usage
        """Check GPU availability and identify conflicts. Uses configurable GPU memory threshold to determine if external processes are using significant GPU resources. Enhanced to detect child processes from lifecycle-aware plugins."""
        try:
            from cjm_fasthtml_sysmon.monitors.gpu import get_gpu_info
        except ImportError:
            # If sysmon not available, return empty conflict
            return ResourceConflict(
                resource_type=ResourceType.GPU_MEMORY,
                status=ResourceStatus.AVAILABLE,
                app_pids=[],
                external_pids=[],
                app_processes=[],
                external_processes=[]
            )

        gpu_info = get_gpu_info()

        if not gpu_info.get('available', False):
            # No GPU available
            return ResourceConflict(
                resource_type=ResourceType.GPU_MEMORY,
                status=ResourceStatus.AVAILABLE,
                app_pids=[],
                external_pids=[],
                app_processes=[],
                external_processes=[]
            )

        # Get GPU details for memory calculations
        gpu_details = gpu_info.get('details', {})
        total_memory_mb = 0
        if gpu_details:
            # Get first GPU (assuming single GPU setup)
            first_gpu = list(gpu_details.values())[0] if gpu_details else {}
            total_memory_mb = first_gpu.get('memory_total', 0)

        # Get all processes using GPU
        gpu_processes = gpu_info.get('processes', [])
        # Get all app PIDs including children
        app_pids = self.get_all_app_pids_including_children()

        app_gpu_pids = []
        external_gpu_pids = []
        app_gpu_processes = []
        external_gpu_processes = []

        for proc in gpu_processes:
            pid = proc.get('pid')
            gpu_memory_mb = proc.get('gpu_memory_mb', 0)

            # Calculate percentage of total GPU memory used by this process
            memory_percent = 0
            if total_memory_mb > 0:
                memory_percent = (gpu_memory_mb / total_memory_mb) * 100

            if pid in app_pids:
                app_gpu_pids.append(pid)
                # Add parent info if this is a child process
                parent_pid = self._child_to_parent.get(pid)
                proc_info = dict(proc)
                if parent_pid:
                    parent_worker = self._worker_states.get(parent_pid)
                    proc_info['parent_pid'] = parent_pid
                    proc_info['parent_worker_type'] = parent_worker.worker_type if parent_worker else None
                app_gpu_processes.append(proc_info)
            else:
                # Only count as external conflict if using significant memory
                if memory_percent >= self.gpu_memory_threshold_percent:
                    external_gpu_pids.append(pid)
                    external_gpu_processes.append(proc)

        # Determine status
        if not app_gpu_pids and not external_gpu_pids:
            status = ResourceStatus.AVAILABLE
        elif app_gpu_pids and not external_gpu_pids:
            status = ResourceStatus.APP_BUSY
        elif external_gpu_pids:
            # External processes using significant memory take precedence
            status = ResourceStatus.EXTERNAL_BUSY
        else:
            status = ResourceStatus.AVAILABLE

        return ResourceConflict(
            resource_type=ResourceType.GPU_MEMORY,
            status=status,
            app_pids=app_gpu_pids,
            external_pids=external_gpu_pids,
            app_processes=app_gpu_processes,
            external_processes=external_gpu_processes
        )

    def check_memory_availability(
        self,
        threshold_percent:float=90.0 # Memory usage threshold to consider as conflict
    ) -> ResourceConflict: # ResourceConflict with details about memory usage
        """Check system memory availability."""
        try:
            from cjm_fasthtml_sysmon.monitors.processes import get_process_info
        except ImportError:
            # If sysmon not available, return empty conflict
            return ResourceConflict(
                resource_type=ResourceType.SYSTEM_MEMORY,
                status=ResourceStatus.AVAILABLE,
                app_pids=[],
                external_pids=[],
                app_processes=[],
                external_processes=[]
            )

        # Get top memory processes
        proc_info = get_process_info(top_n=20)
        top_memory = proc_info.get('top_memory', [])

        # Get all app PIDs including children
        app_pids = self.get_all_app_pids_including_children()

        app_mem_pids = []
        external_mem_pids = []
        app_mem_processes = []
        external_mem_processes = []

        for proc in top_memory:
            pid = proc.get('pid')
            if pid in app_pids:
                app_mem_pids.append(pid)
                app_mem_processes.append(proc)
            else:
                external_mem_pids.append(pid)
                external_mem_processes.append(proc)

        # Simple status: if we have app processes in top memory users
        if app_mem_pids:
            status = ResourceStatus.APP_BUSY
        else:
            status = ResourceStatus.AVAILABLE

        return ResourceConflict(
            resource_type=ResourceType.SYSTEM_MEMORY,
            status=status,
            app_pids=app_mem_pids,
            external_pids=external_mem_pids,
            app_processes=app_mem_processes,
            external_processes=external_mem_processes
        )

### Example: Basic Usage

In [ ]:
# Create a resource manager
manager = ResourceManager(gpu_memory_threshold_percent=45.0)

# Register a worker
manager.register_worker(
    pid=12345,
    worker_type="transcription",
    plugin_name="whisper",
    config={"model_id": "whisper-large-v3"}
)

# Get worker info
worker = manager.get_worker_by_pid(12345)
print(f"Worker type: {worker.worker_type}")
print(f"Plugin: {worker.plugin_name}")
print(f"Status: {worker.status}")

# Update worker state
manager.update_worker_state(12345, status="running", job_id="job-123")
worker = manager.get_worker_by_pid(12345)
print(f"Updated status: {worker.status}")
print(f"Job ID: {worker.job_id}")

# Get worker by job
worker_by_job = manager.get_worker_by_job("job-123")
print(f"Worker for job-123: PID {worker_by_job.pid}")

# Check active worker types
print(f"Active worker types: {manager.get_active_worker_types()}")

# Unregister worker
manager.unregister_worker(12345)
print(f"Workers after unregister: {len(manager.get_all_workers())}")

Worker type: transcription
Plugin: whisper
Status: idle
Updated status: running
Job ID: job-123
Worker for job-123: PID 12345
Active worker types: {'transcription'}
Workers after unregister: 0


In [ ]:
# Example: Mock lifecycle-aware plugin
class MockVLLMPlugin:
    """Simulates a vLLM plugin that spawns child processes."""
    def get_execution_mode(self):
        from enum import Enum
        class MockMode(Enum):
            SUBPROCESS = "subprocess"
        return MockMode.SUBPROCESS
    
    def get_child_pids(self):
        return [99001, 99002, 99003]  # Mock child PIDs
    
    def get_managed_resources(self):
        return {
            'server_url': 'http://localhost:8000',
            'is_running': True
        }
    
    def force_cleanup(self):
        pass

# Register worker with lifecycle-aware plugin
manager = ResourceManager()
mock_plugin = MockVLLMPlugin()

# The manager will automatically detect child processes
manager.register_worker(
    pid=12345,
    worker_type="transcription",
    plugin_id="transcription_voxtral_vllm",
    plugin_name="voxtral_vllm",
    plugin_instance=mock_plugin
)

worker = manager.get_worker_by_pid(12345)
print(f"Worker PID: {worker.pid}")
print(f"Execution mode: {worker.execution_mode}")
print(f"Child PIDs: {worker.child_pids}")
print(f"All related PIDs: {manager.get_all_related_pids(12345)}")
print(f"All app PIDs (including children): {sorted(manager.get_all_app_pids_including_children())}")

Worker PID: 12345
Execution mode: subprocess
Child PIDs: [99001, 99002, 99003]
All related PIDs: [12345, 99001, 99002, 99003]
All app PIDs (including children): [12345, 99001, 99002, 99003]


### Example: Enhanced Usage with Lifecycle/Cloud Plugins

When using plugins from `cjm-fasthtml-plugins`, the resource manager automatically detects and tracks:
- Child processes spawned by plugins
- Cloud/remote resources
- Container IDs and conda environments

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()